Modèle sans convolution (réseau de neurones classique) sur le dataset du Joueur haut - Henri, Minh, Maxime
- Coup droit
- Déplacement
- Revers
- Service

In [53]:
import numpy as np
import cv2
import os
from smart.video import Video, Image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
import tensorflow as tf
import random
from sklearn.preprocessing import LabelEncoder

In [2]:
# Configuration
path_dataset = 'img/JHaut'

# Coup droit, revers, déplacement

## Prises des frames et des vidéos

In [3]:
list_videos = []

In [4]:
y = []
all_paths = []
for dirpath, dirnames, _ in os.walk(path_dataset):
    for dir_type in dirnames:
        
        path_fol = os.path.join(dirpath, dir_type)
        for file in os.listdir(path_fol):
            path_fol_img = os.path.join(path_fol, file)
            if os.path.isdir(path_fol_img) and file == 'images':
                frames = []
                for file_img in os.listdir(path_fol_img):
                    path_img = os.path.join(path_fol_img, file_img)
                    img_obj = Image.load_image(cv2.IMREAD_GRAYSCALE, path_img)
                    img = img_obj.img
                    frames.append(img)	
                if (len(frames) == 15):
                    output_res = path_img.split('\\')[1]
                    all_paths.append(path_fol_img)
                    vid = Video.read_video_from_frames(frames)
                    list_videos.append(vid)
                    y.append(output_res)

In [5]:
list_videos

In [6]:
# Output sans label encoder
y

['coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'coup droit',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'deplacement',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'revers',
 'service',
 'service',
 'service',
 'service',
 'service',
 'service',
 'service',
 'service',
 'service',
 'service',
 'service',
 'service',
 'service']

In [7]:
# One-hot encoding
encoder = LabelEncoder()

Transformer output y en one hot vector
- 0: coup droit
- 1: déplacement
- 2: revers
- 3: service

In [8]:
y = encoder.fit_transform(y)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3], dtype=int64)

In [9]:
output_y = np.unique(y)
output_y

array([0, 1, 2, 3], dtype=int64)

In [10]:
# Résolution des images
input_shape_model = list_videos[0].frames.shape[0] * list_videos[0].frames.shape[1] * list_videos[0].frames.shape[2]
input_shape_model

37500

In [11]:
len(list_videos)

67

In [12]:
train_size = int(len(list_videos) * 0.8)
test_size = len(list_videos) - train_size

In [13]:
## Split video train, test
random.seed(42)
vids_train = random.sample(list_videos, k = train_size)
vids_test = [vid for vid in list_videos if vid not in vids_train]
y_train = [y[i] for i in range(len(y)) if list_videos[i] in vids_train]
y_test = [y[i] for i in range(len(y)) if list_videos[i] in vids_test]
paths_test = [all_paths[i] for i in range(len(all_paths)) if list_videos[i] in vids_test]
len(vids_train), len(y_train), len(vids_test), len(y_test)

(53, 53, 14, 14)

In [14]:
paths_test

['img/JHaut\\coup droit\\AS_18386\\images',
 'img/JHaut\\coup droit\\AS_29198\\images',
 'img/JHaut\\coup droit\\car5493-5847138\\images',
 'img/JHaut\\deplacement\\17694-1786278\\images',
 'img/JHaut\\deplacement\\1980-2563441\\images',
 'img/JHaut\\deplacement\\28040-28230129\\images',
 'img/JHaut\\deplacement\\44260-4450099\\images',
 'img/JHaut\\deplacement\\6481-660086\\images',
 'img/JHaut\\deplacement\\AS_15180\\images',
 'img/JHaut\\revers\\AS_34270\\images',
 'img/JHaut\\revers\\car5936-6390142\\images',
 'img/JHaut\\revers\\car5936-6390257\\images',
 'img/JHaut\\service\\AS_19185\\images',
 'img/JHaut\\service\\AS_35549\\images']

In [15]:
y_train = np.array(y_train, dtype=int)
y_test = np.array(y_test, dtype=int)
y_train

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3])

In [16]:
# Split input X en train, test set
X_train = np.zeros((train_size, 50*50*15), dtype=int)
for i, vid in enumerate(vids_train):
    seq_img = vid.frames.flatten()
    X_train[i] = seq_img
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [17]:
X_test = np.zeros((test_size, 50*50*15), dtype=int)
for i, vid in enumerate(vids_test):
    seq_img = vid.frames.flatten()
    X_test[i] = seq_img
X_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [18]:
# Normalisation
X_train = X_train / 255.0
X_test = X_test / 255.0

## Modèle de réseau de neurones avec classification multiclass

In [19]:
tf.random.set_seed(1234) # for consistent results
model = Sequential(
    [               
        tf.keras.Input(shape = input_shape_model), # 50*50 * 15 = 37500
        Dense(units=16, activation="relu"),
        Dense(units=32, activation="relu"),
        Dense(units=64, activation="sigmoid"),
        Dense(units=128, activation="relu"),
        Dense(units=len(output_y), activation="softmax"),
    ]
)          
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                600016    
                                                                 
 dense_1 (Dense)             (None, 32)                544       
                                                                 
 dense_2 (Dense)             (None, 64)                2112      
                                                                 
 dense_3 (Dense)             (None, 128)               8320      
                                                                 
 dense_4 (Dense)             (None, 4)                 516       
                                                                 
Total params: 611,508
Trainable params: 611,508
Non-trainable params: 0
_________________________________________________________________


In [20]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [21]:
model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = Adam(learning_rate=0.001), # 'Adam',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    epochs = 70,
    batch_size = 32, # données transmises pour une session
    validation_split=0.2,
    callbacks=callbacks_list
)

Epoch 1/70
1/2 [==============>...............] - ETA: 0s - loss: 1.6046 - accuracy: 0.0000e+00
Epoch 1: val_loss improved from inf to 1.59683, saving model to Weights-001--1.59683.hdf5
2/2 [==============================] - 1s 422ms/step - loss: 1.5694 - accuracy: 0.0714 - val_loss: 1.5968 - val_accuracy: 0.0000e+00
Epoch 2/70
1/2 [==============>...............] - ETA: 0s - loss: 1.3235 - accuracy: 0.2812
Epoch 2: val_loss did not improve from 1.59683
2/2 [==============================] - 0s 85ms/step - loss: 1.3050 - accuracy: 0.2857 - val_loss: 2.2099 - val_accuracy: 0.0000e+00
Epoch 3/70
1/2 [==============>...............] - ETA: 0s - loss: 1.1585 - accuracy: 0.3750
Epoch 3: val_loss did not improve from 1.59683
2/2 [==============================] - 0s 71ms/step - loss: 1.1321 - accuracy: 0.4524 - val_loss: 2.7788 - val_accuracy: 0.0000e+00
Epoch 4/70
1/2 [==============>...............] - ETA: 0s - loss: 1.0700 - accuracy: 0.4375
Epoch 4: val_loss did not improve from 1.59683


In [22]:
model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 26ms/step - loss: 2.2362 - accuracy: 0.6429


[2.23620867729187, 0.6428571343421936]

In [23]:
pred = model.predict(X_test)
pred

1/1 [==============================] - 0s 107ms/step


array([[9.8574126e-01, 1.2261490e-02, 1.7229455e-03, 2.7428952e-04],
       [5.2467215e-01, 1.2538719e-02, 4.6138632e-01, 1.4028152e-03],
       [7.6203406e-01, 2.0797512e-01, 2.8665708e-02, 1.3251211e-03],
       [2.7888781e-01, 6.2948935e-02, 6.5671915e-01, 1.4441023e-03],
       [2.8552525e-02, 9.6941400e-01, 1.8100357e-03, 2.2342656e-04],
       [1.8907211e-03, 9.9731928e-01, 7.2984851e-04, 6.0192950e-05],
       [1.3463103e-02, 9.8484784e-01, 1.5306365e-03, 1.5834186e-04],
       [3.8654558e-02, 9.5439738e-01, 6.5933410e-03, 3.5473250e-04],
       [7.8976661e-01, 2.3486086e-03, 2.0703013e-01, 8.5468299e-04],
       [1.3633434e-01, 2.6912370e-01, 5.9277445e-01, 1.7674989e-03],
       [1.9269371e-02, 9.7248411e-01, 7.8756493e-03, 3.7085830e-04],
       [3.7321991e-01, 3.3543635e-02, 5.9172714e-01, 1.5093855e-03],
       [9.3530631e-03, 9.7396201e-01, 1.6385380e-02, 2.9956110e-04],
       [1.5364959e-02, 9.2454821e-01, 5.9508868e-02, 5.7800027e-04]],
      dtype=float32)

In [24]:
y_pred = np.argmax(pred, axis=1)
print("Predicted labels: ", y_pred)
print("True labels:      ", y_test)

Predicted labels:  [0 0 0 2 1 1 1 1 0 2 1 2 1 1]
True labels:       [0 0 0 1 1 1 1 1 1 2 2 2 3 3]


**Accuracy: $9/14$ ($64.28 \%$)**

# Réseaux de neurones convolutifs

## Prises des frames et des vidéos

In [54]:
list_videos_cnn = []
y_cnn = []
for dirpath, dirnames, _ in os.walk(path_dataset):
    for dir_type in dirnames:
        
        path_fol = os.path.join(dirpath, dir_type)
        for file in os.listdir(path_fol):
            path_fol_img = os.path.join(path_fol, file)
            if os.path.isdir(path_fol_img) and file == 'images':
                frames = []
                for file_img in os.listdir(path_fol_img):
                    path_img = os.path.join(path_fol_img, file_img)
                    img_obj = Image.load_image(cv2.IMREAD_COLOR, path_img)
                    img = img_obj.img
                    frames.append(img)	
                if (len(frames) == 15):
                    output_res = path_img.split('\\')[1]
                    vid = Video.read_video_from_frames(frames)
                    list_videos_cnn.append(vid)
                    y_cnn.append(output_res)

In [55]:
list_videos_cnn[0].frames.shape[1:]

(50, 50, 3)

In [56]:
# One-hot encoding
encoder = LabelEncoder()

Transformer output y en one hot vector
- 0: coup droit
- 1: déplacement
- 2: revers
- 3: service

In [57]:
y_cnn = encoder.fit_transform(y_cnn)
y_cnn

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3], dtype=int64)

In [58]:
output_y = np.unique(y_cnn)
output_y

array([0, 1, 2, 3], dtype=int64)

In [59]:
len(list_videos_cnn)

67

In [60]:
train_size = int(len(list_videos_cnn) * 0.8)
test_size = len(list_videos_cnn) - train_size

In [61]:
## Split video train, test
random.seed(42)
vids_train_cnn = random.sample(list_videos_cnn, k = train_size)
vids_test_cnn = [vid for vid in list_videos_cnn if vid not in vids_train_cnn]
y_train_cnn = [y_cnn[i] for i in range(len(y_cnn)) if list_videos_cnn[i] in vids_train_cnn]
y_test_cnn = [y_cnn[i] for i in range(len(y_cnn)) if list_videos_cnn[i] in vids_test_cnn]
len(vids_train_cnn), len(y_train_cnn), len(vids_test_cnn), len(y_test_cnn)

(53, 53, 14, 14)

In [62]:
y_train_cnn = np.array(y_train_cnn, dtype=int)
y_test_cnn = np.array(y_test_cnn, dtype=int)
y_train_cnn

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3])

In [63]:
# Split input X en train, test set
X_train_cnn = np.zeros((train_size, 50, 15*50, 3), dtype=int)
for i, vid in enumerate(vids_train_cnn):
    for j, frame in enumerate(vid.frames):
        X_train_cnn[i, :, 50*j:50*(j+1), :] = frame
X_train_cnn.shape

(53, 50, 750, 3)

In [64]:
X_test_cnn = np.zeros((test_size, 50, 15*50, 3), dtype=int)
for i, vid in enumerate(vids_test_cnn):
    for j, frame in enumerate(vid.frames):
        X_test_cnn[i, :, 50*j:50*(j+1), :] = frame
X_test_cnn.shape

(14, 50, 750, 3)

In [65]:
input_shape_cnn = X_train_cnn.shape[1:]
input_shape_cnn

(50, 750, 3)

## Modèle de réseau de neurones convolutifs

In [74]:
model_cnn = Sequential(
    [
        tf.keras.Input(shape = input_shape_cnn), # 50, 15*50, 3
        Conv2D(filters=16, kernel_size=(3, 3), activation="relu"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),
        
        Conv2D(filters=32, kernel_size=(3, 3), activation="relu"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),
        
        Conv2D(filters=64, kernel_size=(3, 3), activation="relu"),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),
        
        Flatten(),
        Dense(units=128, activation="relu"),
        Dense(units=len(output_y), activation="softmax"),
    ]
)

model_cnn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 48, 748, 16)       448       
                                                                 
 batch_normalization_2 (Batc  (None, 48, 748, 16)      64        
 hNormalization)                                                 
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 24, 374, 16)      0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 24, 374, 16)       0         
                                                                 
 conv2d_7 (Conv2D)           (None, 22, 372, 32)       4640      
                                                                 
 batch_normalization_3 (Batc  (None, 22, 372, 32)     

In [75]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [76]:
model_cnn.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer='rmsprop',
    metrics=['accuracy']
)

history_cnn = model_cnn.fit(
    X_train_cnn, y_train_cnn,
    epochs = 70,
    batch_size = 32, # données transmises pour une session
    validation_split=0.2,
    callbacks=callbacks_list
)

Epoch 1/70
2/2 [==============================] - ETA: 0s - loss: 31.8355 - accuracy: 0.2143
Epoch 1: val_loss improved from inf to 750.79340, saving model to Weights-001--750.79340.hdf5
2/2 [==============================] - 3s 688ms/step - loss: 31.8355 - accuracy: 0.2143 - val_loss: 750.7934 - val_accuracy: 0.0000e+00
Epoch 2/70
2/2 [==============================] - ETA: 0s - loss: 46.3705 - accuracy: 0.3095
Epoch 2: val_loss improved from 750.79340 to 273.56985, saving model to Weights-002--273.56985.hdf5
2/2 [==============================] - 1s 361ms/step - loss: 46.3705 - accuracy: 0.3095 - val_loss: 273.5699 - val_accuracy: 0.0000e+00
Epoch 3/70
2/2 [==============================] - ETA: 0s - loss: 9.9025 - accuracy: 0.4286
Epoch 3: val_loss improved from 273.56985 to 178.38153, saving model to Weights-003--178.38153.hdf5
2/2 [==============================] - 1s 364ms/step - loss: 9.9025 - accuracy: 0.4286 - val_loss: 178.3815 - val_accuracy: 0.0000e+00
Epoch 4/70
2/2 [=====

In [77]:
model_cnn.evaluate(X_test_cnn, y_test_cnn)

1/1 [==============================] - 0s 157ms/step - loss: 5.5830 - accuracy: 0.6429


[5.582958221435547, 0.6428571343421936]

In [78]:
pred_cnn = model_cnn.predict(X_test_cnn)
pred_cnn

1/1 [==============================] - 0s 179ms/step


array([[1.08550958e-01, 8.01857352e-01, 8.95917043e-02, 1.14367316e-12],
       [4.77968931e-01, 4.47016247e-02, 4.77329373e-01, 1.39485439e-14],
       [9.88745511e-01, 1.14223389e-04, 1.11402879e-02, 2.67722368e-13],
       [7.28632789e-03, 3.22381267e-04, 9.92391288e-01, 4.04613282e-15],
       [1.47412986e-01, 8.24901462e-01, 2.76854690e-02, 4.01182858e-13],
       [5.56604355e-04, 9.99277174e-01, 1.66233018e-04, 8.60163652e-15],
       [3.28397029e-04, 9.99657393e-01, 1.42357785e-05, 8.07357328e-13],
       [1.25613108e-01, 8.69975448e-01, 4.41146642e-03, 2.36222847e-11],
       [2.58365601e-01, 1.72181274e-07, 7.41634190e-01, 5.19251261e-16],
       [1.42059755e-03, 5.41938562e-03, 9.93160009e-01, 8.61281532e-14],
       [1.01163588e-01, 3.33228469e-01, 5.65606475e-01, 1.48599213e-06],
       [4.72360343e-01, 3.68919609e-06, 5.27635932e-01, 6.31018624e-15],
       [1.71187625e-03, 3.46972817e-03, 9.94818389e-01, 4.20311312e-12],
       [6.44465536e-03, 3.87802184e-01, 6.05753183e

In [79]:
y_pred_cnn = np.argmax(pred_cnn, axis=1)
print("Predicted labels: ", y_pred_cnn)
print("True labels:      ", y_test_cnn)

Predicted labels:  [1 0 0 2 1 1 1 1 2 2 2 2 2 2]
True labels:       [0 0 0 1 1 1 1 1 1 2 2 2 3 3]


**Accuracy: $9 / 14$ ($64.28 \%$)**